In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})

In [3]:
rat=pd.read_csv('ratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [4]:
df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [5]:
rat.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
df=pd.merge(rat,df,on='movieId')

In [7]:
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [9]:
clean=df.dropna(axis=0,subset=['title'])

In [35]:
res= (clean.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )

In [36]:
res.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [37]:
rwtc = df.merge(res, left_on = 'title', right_on = 'title', how = 'left')
rwtc.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [38]:
popthresh=50

In [39]:
fin=rwtc.query('totalRatingCount>=@popthresh')

In [41]:
fin.shape

(41362, 5)

# Creating a Pivot Matrix

In [44]:
feat=fin.pivot_table(index='title',columns='userId',values='rating').fillna(0)

In [45]:
feat.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [46]:
from scipy.sparse import csr_matrix

fin_df = csr_matrix(feat.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(fin_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [57]:
q= np.random.choice(feat.shape[0])
print(q)
d, ind = model_knn.kneighbors(feat.iloc[q,:].values.reshape(1, -1), n_neighbors = 6)
for i in range(0, len(d.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(feat.index[q]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i,feat.index[indices.flatten()[i]], d.flatten()[i]))

322
Recommendations for Raising Arizona (1987):

1: Ferris Bueller's Day Off (1986), with distance of 0.47745078802108765:
2: Caddyshack (1980), with distance of 0.48472416400909424:
3: Hook (1991), with distance of 0.4866028428077698:
4: Dogma (1999), with distance of 0.48909300565719604:
5: Who Framed Roger Rabbit? (1988), with distance of 0.49472469091415405:
